In [ ]:
# 1.1 - Exploración inicial de los datos (EDA)
# Se identifican valores nulos, atípicos y datos faltantes en las columnas relevantes.

import pandas as pd  # Manejo de datos
import numpy as np  # Cálculos numéricos
import matplotlib.pyplot as plt  # Gráficos básicos
import seaborn as sns  # Visualización avanzada

# Configuración para ver todas las columnas en la salida
pd.set_option('display.max_columns', None)

In [ ]:
## Fase 1: Exploración y Limpieza

In [ ]:
# 1.2 - Carga de datos en DataFrames
# Se cargan los datasets en dataframes para su análisis
try:
    df_flights = pd.read_csv("Customer Flight Activity.csv")
    df_customers = pd.read_csv("Customer Loyalty History.csv")
    print("Datos cargados correctamente.")
except FileNotFoundError:
    print("Error: No se encontraron los archivos.")

# Verificamos las primeras filas de cada dataset
print("\nPrimeras filas de df_flights:")
display(df_flights.head())

print("\nPrimeras filas de df_customers:")
display(df_customers.head())

# Dos archivos CSV: uno con la actividad de vuelos y otro con los datos de clientes.

In [ ]:
print("Número exacto de filas en df_flights:", len(df_flights))
print("Número exacto de filas en df_customers:", len(df_customers))

In [ ]:
# 1.3 - Exploración inicial y unión de los datasets
# Se obtiene información general de cada dataset para identificar problemas potenciales

def exploracion_general(df, nombre):
    print(f"\n--- Exploración de {nombre} ---")
    print("\nResumen estadístico de variables numéricas:")
    print(df.describe())
    print("\nTipos de datos por columna:")
    print(df.dtypes)
    print("\nCantidad de valores nulos:")
    print(df.isnull().sum())
    print("\nPorcentaje de valores nulos por columna:")
    print((df.isnull().sum() / len(df) * 100).round(2))
    print("\nCantidad de valores duplicados:")
    print(df.duplicated().sum())

# Aplicamos la función a ambos datasets
exploracion_general(df_flights, "df_flights")
exploracion_general(df_customers, "df_customers")

# Mensaje indicando que la exploración ha finalizado
print(" Exploración inicial completada.")

# Eliminar duplicados antes de unir los datasets
df_flights.drop_duplicates(inplace=True)
df_customers.drop_duplicates(inplace=True)
print(" Duplicados eliminados correctamente.")

# Unir los datasets de la forma más eficiente
# Se combinan los datos de vuelos y clientes usando "Loyalty Number" como clave

df_merged = df_flights.merge(df_customers, on="Loyalty Number", how="left")
print("Datos combinados correctamente.")


In [ ]:
# 1.2.1 - Tratamiento de valores nulos
# Se verifican y manejan los valores nulos en columnas clave

print("\n Valores nulos por columna antes del tratamiento:")
print(df_merged.isnull().sum()[df_merged.isnull().sum() > 0])  # Solo muestra columnas con nulos
print("\n Porcentaje de valores nulos por columna:")
print((df_merged.isnull().sum()[df_merged.isnull().sum() > 0] / len(df_merged) * 100).round(2))

# Imputamos valores nulos en 'Salary' con la media, ya que la distribución parece normal
df_merged["Salary"].fillna(df_merged["Salary"].mean(), inplace=True)

# Eliminamos columnas con demasiados valores nulos que no aportan al análisis
df_merged.drop(columns=["Cancellation Year", "Cancellation Month"], inplace=True)

print("\n Valores nulos después del tratamiento:")
print(df_merged.isnull().sum()[df_merged.isnull().sum() > 0])  # Verificamos si aún quedan nulos
print("Tratamiento de valores nulos completado.")

In [ ]:
# 1.2.2 - Verificación de consistencia de los datos
# Se revisan valores únicos en variables categóricas y rangos en numéricas

print("\n Categorías en 'Education':", df_merged["Education"].unique())
print("\n Categorías en 'Marital Status':", df_merged["Marital Status"].unique())
print("\n Rango de salarios:", df_merged["Salary"].min(), "-", df_merged["Salary"].max())
print("\n Rango de CLV:", df_merged["CLV"].min(), "-", df_merged["CLV"].max())   # CLV (Customer Lifetime Value)

# Buscamos valores atípicos en Salary (valores negativos)
outliers_salary = df_merged[df_merged["Salary"] < 0]
print(f"\n Valores atípicos en Salary: {len(outliers_salary)} encontrados.")

# Si existen valores negativos en Salary, los convertimos a positivos
if len(outliers_salary) > 0:
    df_merged["Salary"] = df_merged["Salary"].abs()
    print("Valores negativos en Salary corregidos.")

print("Verificación de consistencia de los datos completada.")


In [ ]:
# 1.2.3 - Ajuste de tipos de datos
# Convertimos 'Enrollment Year' y 'Enrollment Month' en una fecha completa

df_merged["Enrollment Date"] = pd.to_datetime(
    df_merged["Enrollment Year"].astype(str) + "-" + df_merged["Enrollment Month"].astype(str),
    errors="coerce"
)

print("\n Ajuste de tipos de datos completado.")
print(df_merged.dtypes)  # Verificamos cambios

In [ ]:
# 1.2.4 - Eliminación de columnas innecesarias
# Eliminamos columnas que ya no son útiles después de la transformación

df_merged.drop(columns=["Enrollment Year", "Enrollment Month"], inplace=True)
print("Columnas innecesarias eliminadas.")

In [ ]:
# 1.2.5 - Comprobación final de los datos limpios
# Revisamos que todo esté en orden tras la limpieza

print("\n Resumen final de los datos limpios:")
print(df_merged.info())

print("\n Primeras filas de los datos limpios:")
print(df_merged.head())

print("\n Estadísticas finales tras la limpieza:")
print(df_merged.describe())

print("Limpieza de datos completada con éxito.")


In [ ]:
## Fase 2: Visualización

In [ ]:
# 2.1 - Distribución de la cantidad de vuelos reservados por mes durante cada año

# Crear un DataFrame con la cantidad total de vuelos reservados por mes y año
df_year_month = df_merged.groupby(["Year", "Month"])["Flights Booked"].sum().reset_index()

# Configuración del gráfico de líneas
plt.figure(figsize=(9, 5))
sns.lineplot(x="Month", 
             y="Flights Booked", 
             hue="Year",  # Diferenciar por año
             palette=["#87CEEB", "#4169E1"], 
             data=df_year_month)

# Personalización del gráfico
plt.title("Vuelos reservados por mes en los años 2017 y 2018", fontsize=14)
plt.xticks(ticks=range(1, 13), labels=["Ene", "Feb", "Mar", "Abr", "May", "Jun", "Jul", "Ago", "Sep", "Oct", "Nov", "Dic"], rotation=45)
plt.xlabel("Mes", fontsize=12)
plt.ylabel("Vuelos Reservados", fontsize=12)
plt.grid(True)

plt.show()

# Configuración del gráfico de barras
plt.figure(figsize=(9, 5))

# Ordenar correctamente los datos para asegurar la alineación correcta
df_year_month_sorted = df_year_month.sort_values(by=["Month"])

# Crear gráfico de barras asegurando que los meses aparecen en orden correcto
sns.barplot(x="Month", 
            y="Flights Booked", 
            hue="Year",  
            data=df_year_month_sorted, 
            palette=["#87CEEB", "#4169E1"],  
            estimator=sum)

# Personalización del gráfico
plt.title("Vuelos reservados por mes en los años 2017 y 2018", fontsize=14)
plt.xticks(ticks=range(12), labels=["Ene", "Feb", "Mar", "Abr", "May", "Jun", "Jul", "Ago", "Sep", "Oct", "Nov", "Dic"], rotation=55)
plt.xlabel("Mes", fontsize=12)
plt.ylabel("Vuelos Reservados", fontsize=12)
plt.grid(axis="y", linestyle="--", alpha=0.7)

plt.show()

📊  Justificación del uso de gráficos:
	•	Línea → Se utiliza para visualizar la tendencia de vuelos reservados a lo largo del tiempo, ya que permite identificar aumentos y caídas en la demanda de forma clara.
	•	Barras → Se usa para comparar de manera más intuitiva la cantidad de vuelos por mes en cada año, ayudando a ver diferencias absolutas.

🔍 	Tendencia estacional:
	•	Hay un pico en los meses de mayo a julio, lo que indica que estos meses tienen una mayor demanda de vuelos.
	•	Tras este pico, la cantidad de vuelos disminuye en los meses siguientes, especialmente en septiembre y octubre.
	Comparación entre años:
	•	2018 muestra un incremento general en la cantidad de vuelos reservados en comparación con 2017.
	•	Sin embargo, la tendencia sigue un patrón similar en ambos años, lo que sugiere una estacionalidad en la demanda.

In [ ]:
# 2.2 - Relación entre la distancia de los vuelos y los puntos acumulados

plt.figure(figsize=(9, 5))

# Crear scatterplot
sns.scatterplot(x="Distance", 
                y="Points Accumulated", 
                data=df_merged, 
                alpha=0.5,  # Transparencia para visualizar mejor la densidad
                color="mediumturquoise")  

# Personalización del gráfico
plt.title("Relación entre la distancia de los vuelos y los puntos acumulados", fontsize=14)
plt.xlabel("Distancia del vuelo (km)", fontsize=12)
plt.ylabel("Puntos acumulados", fontsize=12)
plt.grid(True, linestyle="--", alpha=0.7)

plt.show()

📊  Se utiliza un scatterplot porque queremos analizar la relación entre dos variables numéricas: la distancia recorrida y los puntos acumulados. Este tipo de gráfico nos permite visualizar si hay una tendencia clara entre ambas variables y detectar posibles valores atípicos.

🔍 Se observa que a mayor distancia recorrida, mayor cantidad de puntos acumulados. Esto indica que los puntos se otorgan en función de la distancia del vuelo, lo que tiene sentido en un programa de fidelización de aerolíneas.


In [ ]:
# 2.3 - Distribución de clientes por provincia

# Eliminamos duplicados para contar clientes únicos
df_unique_clients = df_merged.drop_duplicates(subset="Loyalty Number")

# Contamos la cantidad de clientes únicos por provincia
clientes_por_provincia = df_unique_clients["Province"].value_counts()

# Visualización de la distribución de clientes por provincia
plt.figure(figsize=(11, 5))
sns.barplot(x=clientes_por_provincia.index, 
            y=clientes_por_provincia.values, 
            hue=clientes_por_provincia.index,  # Añadimos hue para evitar el warning
            palette="viridis")  # Se usa una paleta de colores con suficientes tonos

# Ajustes visuales
plt.title("Cantidad de clientes únicos por provincia", fontsize=14)
plt.xlabel("Provincia", fontsize=12, labelpad=10)  # Ajuste del padding del xlabel
plt.ylabel("Número de clientes", fontsize=12)

# Rotamos las etiquetas y las alineamos hacia la izquierda
plt.xticks(rotation=45, ha="right")  # Alinear etiquetas a la derecha para que se vean más a la izquierda
plt.grid(axis="y", linestyle="--", alpha=0.7)

plt.show()

📊 	•	Permite comparar de manera clara y ordenada el número de clientes en cada provincia.
	•	Es ideal para variables categóricas, como provincias, en lugar de gráficos de dispersión o circulares que no serían tan efectivos en este caso.

🔍 	•	Algunas provincias tienen una mayor cantidad de clientes en comparación con otras.
	•	Se puede ver que la mayoría de clientes son de Ontario, mientras que la provincia con menor cantidad de clientes es Prince Edward Island.
	•	Esto podría estar influenciado por factores como densidad poblacional, accesibilidad a aeropuertos y nivel económico.
	•	Las provincias con menos clientes podrían indicar menor interés o acceso al programa de fidelización.

In [ ]:
# 2.4 - Comparación del salario promedio por nivel educativo

# Eliminamos duplicados para no contar a la misma persona varias veces
df_unique_clients = df_merged.drop_duplicates(subset="Loyalty Number")

# Calculamos el salario promedio por nivel educativo
salario_por_educacion = df_unique_clients.groupby("Education")["Salary"].mean().sort_values()

# Visualización de la comparación de salario promedio por nivel educativo
plt.figure(figsize=(8, 5))
sns.barplot(x=salario_por_educacion.index, 
            y=salario_por_educacion.values, 
            hue=salario_por_educacion.index,  
            palette="Greens")  # Paleta predefinida que se adapta a cualquier cantidad de categorías

# Ajustes visuales
plt.title("Salario promedio por nivel educativo", fontsize=14)
plt.xlabel("Nivel Educativo", fontsize=12)
plt.ylabel("Salario Promedio", fontsize=12)
plt.xticks(rotation=65)  # Rotamos etiquetas para mejor visibilidad
plt.grid(axis="y", linestyle="--", alpha=0.7)
plt.show()

📊  •   barplot porque es ideal para comparar valores categóricos.
	•	Seaborn barplot permite visualizar fácilmente cómo varía el salario promedio según la educación.
	•	Ordenamos los niveles educativos de menor a mayor salario para facilitar la interpretación.
	•	Se usó la paleta “Greens” para mantener la coherencia de colores con los gráficos anteriores.

🔍  •	Se observa que, a medida que aumenta el nivel educativo, también aumenta el salario promedio de los clientes.
	•	Existen diferencias salariales significativas entre distintos niveles de educación.
	•	Esto sugiere que la educación juega un papel clave en la capacidad adquisitiva de los clientes, lo que puede influir en sus decisiones de viaje.

In [ ]:
# 2.5 - Porcentaje de clientes por tipo de tarjeta de fidelidad

# Eliminamos duplicados para contar cada cliente solo una vez
df_unique_clients = df_merged.drop_duplicates(subset="Loyalty Number")

# Calculamos el porcentaje de clientes por tipo de tarjeta
clientes_por_tarjeta = df_unique_clients["Loyalty Card"].value_counts(normalize=True) * 100

# Definimos colores personalizados
colores = ["lightseagreen", "mediumseagreen", "teal", "turquoise"]

# Creación del gráfico Pie
plt.figure(figsize=(5, 5))
plt.pie(clientes_por_tarjeta, labels=clientes_por_tarjeta.index, autopct="%1.1f%%", colors=colores, startangle=90, wedgeprops={"edgecolor": "black", "linewidth": 1})

# Ajustes visuales
plt.title("Distribución de clientes por tipo de tarjeta de fidelidad", fontsize=14)
plt.show()

📊  •   Este gráfico permite comparar claramente qué tipos de tarjeta son más populares entre los clientes.
    •   Al usar porcentajes en lugar de valores absolutos, podemos evaluar la distribución relativa de los clientes.

🔍 	•	La mayoría de los clientes usan la tarjeta Star, lo que sugiere que es la opción más accesible o con mejores beneficios.
	•	Aurora y Nova tienen una proporción menor de clientes, lo que podría indicar que están dirigidas a segmentos más específicos o con requisitos diferentes.

In [ ]:
# Verificar cuántos tipos de tarjetas existen
print(df_unique_clients["Loyalty Card"].value_counts())

In [ ]:
# 2.6 - Distribución de clientes según estado civil y género

# Eliminamos duplicados para no contar a la misma persona varias veces
df_unique_clients = df_merged.drop_duplicates(subset="Loyalty Number")

# Visualización de la distribución de clientes según estado civil y género
plt.figure(figsize=(8, 5))
sns.countplot(x="Marital Status", hue="Gender", data=df_unique_clients, palette="crest")

# Ajustes visuales
plt.title("Distribución de clientes según estado civil y género", fontsize=14)
plt.xlabel("Estado Civil", fontsize=12)
plt.ylabel("Número de Clientes", fontsize=12)
plt.xticks(rotation=0)  # Mantener etiquetas rectas para este tipo de gráfico
plt.legend(title="Género")
plt.grid(axis="y", linestyle="--", alpha=0.6)  # Grid en el eje Y para facilitar la lectura

plt.show()

📊  •  Este gráfico de barras (countplot) muestra la distribución de clientes según su estado civil, diferenciando entre hombres y mujeres con hue="Gender".

🔍 	•	Las personas casadas son el grupo más numeroso, seguidas por los solteros.
	•	En la categoría de Divorciado, la distribución por género es casi equitativa.

In [ ]:
# Contar la cantidad de clientes por estado civil y género
estado_civil_genero = df_unique_clients.groupby(["Marital Status", "Gender"])["Loyalty Number"].count()

# Mostrar los datos específicamente para "Married"
print(estado_civil_genero.loc["Married"])

In [ ]:
print(df_unique_clients["Marital Status"].value_counts())

In [ ]:
# Creación del DataFrame final limpio
df_final = df_merged.dropna().drop_duplicates(subset="Loyalty Number")

# Verificación de que está limpio
print("\nResumen del DataFrame final limpio:")
print(df_final.info())

print("\nValores nulos en el DataFrame final:")
print(df_final.isnull().sum().sum())  # Debe ser 0

print("\nCantidad de clientes únicos en el DataFrame final:")
print(len(df_final))

# Confirmación de que el DataFrame está listo
print("DataFrame final limpio y listo para la fase 3.")

In [ ]:
## Fase 3: Evaluación de Diferencias en Reservas de Vuelos por Nivel Educativo

In [ ]:
# 3.1 - Preparación de Datos - 	Objetivo: Filtrar únicamente las columnas necesarias para el análisis (‘Flights Booked’ y ‘Education’).

# Filtramos el DataFrame con solo las columnas necesarias
df_education_flights = df_final[["Flights Booked", "Education"]].copy()

# Eliminamos posibles valores nulos en estas columnas
df_education_flights.dropna(inplace=True)

# Mostramos las primeras filas para verificar los datos
df_education_flights.head()

,Flights Booked,Education
0,3,Bachelor
1,10,College
2,6,College
3,0,Bachelor
4,0,Bachelor


In [ ]:
# 3.2 - Análisis Descriptivo - Obtener estadísticas básicas (media, desviación estándar, etc.) para cada nivel educativo.

# Agrupamos por nivel educativo y calculamos estadísticas descriptivas
stats_education = df_education_flights.groupby("Education")["Flights Booked"].agg(["mean", "std", "count"]).round(2)

#por concluir-------------